In [28]:
import random
import os
import json
import numpy as np

In [ ]:
# User defines problem size
num_resources = 3
num_products = 4
num_scenarios = 5

os.makedirs("input_data", exist_ok=True)

resources = [f"r{i+1}" for i in range(num_resources)]
products = [f"p{i+1}" for i in range(num_products)]
scenarios = [f"s{i+1}" for i in range(num_scenarios)]

# Generate random data
data = {}
data['resources'] = resources
data['products'] = products
data['scenarios'] = scenarios

data['cost_x'] = {r: round(random.uniform(1, 10), 2) for r in resources}

data['profit_y'] = {p: round(random.uniform(10, 100), 2) for p in products}

data['usage'] = {r: {p: round(random.uniform(0.5, 10), 2) for p in products} for r in resources}

prob_values = np.random.dirichlet(np.ones(num_scenarios), size=1)[0]
data['p'] = {scenarios[i]: round(prob_values[i], 3) for i in range(num_scenarios)}

data['demand_ub'] = {
    s: {p: random.randint(50, 300) for p in products}
    for s in scenarios
}

with open(os.path.join("input_data", "model_input.txt"), "w") as f:
    json.dump(data, f, indent=2)

print("✔ All random input data saved in 'input_data/model_input.txt'.")

✔ All random input data saved in 'input_data/model_input.txt'.


In [27]:
from gurobipy import Model, GRB

with open(os.path.join("input_data", "model_input.txt"), 'r') as f:
    data = json.load(f)

resources = data['resources']
products = data['products']
scenarios = data['scenarios']
cost_x = data['cost_x']
profit_y = data['profit_y']
usage = data['usage']
p = data['p']
demand_ub = data['demand_ub']

m = Model("General_Two_Stage_Model")

x = m.addVars(resources, lb=0.0, name="x")

y = m.addVars(products, scenarios, lb=0.0, name="y")

first_stage_cost = sum(cost_x[r] * x[r] for r in resources)
recourse_profit = sum(
    p[s] * sum(profit_y[i] * y[i, s] for i in products)
    for s in scenarios
)
m.setObjective(first_stage_cost - recourse_profit, GRB.MINIMIZE)

for s in scenarios:
    for r in resources:
        m.addConstr(
            -x[r] + sum(usage[r][i] * y[i, s] for i in products) <= 0,
            name=f"capacity_{r}_{s}"
        )

for s in scenarios:
    for i in products:
        m.addConstr(
            y[i, s] <= demand_ub[s][i],
            name=f"demand_ub_{i}_{s}"
        )

m.optimize()

if m.status == GRB.OPTIMAL:
    print("\nOptimal First-Stage Decisions:")
    for r in resources:
        print(f"  x[{r}] = {x[r].X:.2f}")

    print("\nOptimal Recourse Decisions (per scenario):")
    for s in scenarios:
        print(f"  Scenario {s}:")
        for i in products:
            print(f"    y[{i}, {s}] = {y[i, s].X:.2f}")

    print(f"\nObjective Value: {m.ObjVal:.2f}")
else:
    print("\nModel did not solve to optimality.")


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 35 rows, 23 columns and 95 nonzeros
Model fingerprint: 0xa16b9175
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e-01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 3e+02]
Presolve removed 20 rows and 0 columns
Presolve time: 0.00s
Presolved: 15 rows, 23 columns, 75 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.1722229e+03   5.823122e+02   0.000000e+00      0s
       4   -8.5490640e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective -8.549064000e+02

Optimal First-Stage Decisions:
  x[r1] = 236.52
  x[r2] = 748.44
  x[r3] = 179.28

Optimal Recourse Decisions (per 